# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [23]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from geoapify_key import geoapi_key

In [24]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data_cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sao joao da barra,-21.6403,-41.0511,80.69,86,89,4.47,BR,2023-04-01 22:45:19
1,1,broome,42.2506,-75.8330,54.70,69,75,18.41,US,2023-04-01 22:45:19
2,2,birjand,32.8663,59.2211,59.47,26,20,4.61,IR,2023-04-01 22:45:19
3,3,kapaa,22.0752,-159.3190,75.52,84,100,13.80,US,2023-04-01 22:45:19
4,4,bluff,-46.6000,168.3333,55.80,67,100,15.59,NZ,2023-04-01 22:45:19


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [25]:
humidity_citymap = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["City", "Humidity"]
)

# Display the map plot
humidity_citymap

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [26]:
# Narrow down cities that fit criteria and drop any results with null values
# desired temperature range
min_temp = 50
max_temp = 90
max_humidity = 60

# desired wind speed range
max_wind_speed = 10

# desired cloudiness range
max_cloudiness = 10
ideal_cities = city_data_df[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp) & 
                            (city_data_df["Wind Speed"] <= max_wind_speed) & (city_data_df["Cloudiness"] <= max_cloudiness) & (city_data_df['Humidity'] <= max_humidity)]

# Drop any rows with null values
clean_ideal_cities = ideal_cities.dropna()

# Display sample data
ideal_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,port augusta,-32.5000,137.7667,59.85,45,0,7.18,AU,2023-04-01 22:45:19
42,42,taoudenni,22.6783,-3.9836,83.43,7,0,9.98,ML,2023-04-01 22:45:23
63,63,ponta do sol,32.6667,-17.1000,65.12,57,0,5.91,PT,2023-04-01 22:45:24
134,134,mehriz,31.5917,54.4318,50.50,22,0,2.35,IR,2023-04-01 22:45:28
175,175,saint george,37.1041,-113.5841,67.42,25,0,4.61,US,2023-04-01 22:45:31
176,176,santa maria,-29.6842,-53.8069,65.50,59,0,3.44,BR,2023-04-01 22:45:31
211,211,lamar,33.6668,-95.5836,71.51,27,0,5.75,US,2023-04-01 22:45:33
256,256,aswan,24.0934,32.9070,65.50,20,0,0.00,EG,2023-04-01 22:45:37
329,329,abnub,27.2696,31.1511,61.02,36,0,7.16,EG,2023-04-01 22:45:42
335,335,norman,35.2226,-97.4395,68.68,28,0,6.91,US,2023-04-01 22:45:42


### Step 3: Create a new DataFrame called `hotel_df`.

In [27]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_ideal_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,port augusta,AU,-32.5000,137.7667,45,
42,taoudenni,ML,22.6783,-3.9836,7,
63,ponta do sol,PT,32.6667,-17.1000,57,
134,mehriz,IR,31.5917,54.4318,22,
175,saint george,US,37.1041,-113.5841,25,
176,santa maria,BR,-29.6842,-53.8069,59,
211,lamar,US,33.6668,-95.5836,27,
256,aswan,EG,24.0934,32.9070,20,
329,abnub,EG,27.2696,31.1511,36,
335,norman,US,35.2226,-97.4395,28,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapi_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port augusta - nearest hotel: Hotel Commonwealth
taoudenni - nearest hotel: No hotel found
ponta do sol - nearest hotel: Estalagem Ponta do Sol
mehriz - nearest hotel: هتل رستوران شاه عباسی
saint george - nearest hotel: The Advenire, an Autograph Collection Hotel
santa maria - nearest hotel: Hotel Morotin
lamar - nearest hotel: Days Inn by Wyndham Paris
aswan - nearest hotel: Yaseen hotel
abnub - nearest hotel: University Guest House
norman - nearest hotel: Days Inn by Wyndham Norman
san vicente - nearest hotel: Los Lapachos
youkounkoun - nearest hotel: No hotel found
atar - nearest hotel: فندق سكليل
araouane - nearest hotel: No hotel found
asyut - nearest hotel: فندق الحسين
cuervos - nearest hotel: No hotel found
nokaneng - nearest hotel: No hotel found
calvinia - nearest hotel: No hotel found
agadez - nearest hotel: No hotel found
murray bridge - nearest hotel: Bridgeport


,City,Country,Lat,Lng,Humidity,Hotel Name
5,port augusta,AU,-32.5000,137.7667,45,Hotel Commonwealth
42,taoudenni,ML,22.6783,-3.9836,7,No hotel found
63,ponta do sol,PT,32.6667,-17.1000,57,Estalagem Ponta do Sol
134,mehriz,IR,31.5917,54.4318,22,هتل رستوران شاه عباسی
175,saint george,US,37.1041,-113.5841,25,"The Advenire, an Autograph Collection Hotel"
176,santa maria,BR,-29.6842,-53.8069,59,Hotel Morotin
211,lamar,US,33.6668,-95.5836,27,Days Inn by Wyndham Paris
256,aswan,EG,24.0934,32.9070,20,Yaseen hotel
329,abnub,EG,27.2696,31.1511,36,University Guest House
335,norman,US,35.2226,-97.4395,28,Days Inn by Wyndham Norman


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [42]:
%%capture --no-display

# Configure the map plot
hotel_citymap = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 400,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha= 0.8,
    hover_cols = ["Country", "Hotel Name", 'City']
)

# Display the map plot
hotel_citymap

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)